<a href="https://colab.research.google.com/github/arnav-exe/Data-Mining-Labs/blob/main/ECS607U_Lab04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab session 4: Data Warehousing and On-line Analytical Processing

## Introduction

The aim of this lab (Lab session 4) is for students to get experience with **Data Warehousing** and **On-line Analytical Processing (OLAP)** covered in lecture 5, and more specifically with the concepts of **data cubes**, **data cube measures**, **typical OLAP operations**, and **data cube computation**.


## 1. Introduction to Cubes

This chapter describes step-by-step how to use Cubes (http://cubes.databrewery.org/), a lightweight Python framework and set of tools for development of reporting and analytical applications, Online Analytical Processing (OLAP), multidimensional analysis and browsing of aggregated data. We will be working with v1.1 of Cubes. Cubes features:
- a logical view of analysed data - how analysts look at data, how they think of data, not not how the data are physically implemented in the data stores
- OLAP and aggregated browsing (default backend is for relational databse - ROLAP)
- hierarchical dimensions (attributes that have hierarchical dependencies, such as category-subcategory or country-region)
- multiple hierarchies in a dimension
- localizable metadata and data (see Localization)
- authentication and authorization of cubes and their data
- pluggable data warehouse – plug-in other cube-like (multidimensional) data sources

Cubes is meant to be used by application builders that want to provide analytical functionality. Cubes also relies on methods from SQLAlchemy (https://www.sqlalchemy.org/), an open-source SQL toolkit and object-relational mapper for Python.

## 2. Data Preparation

The example data used here is the International Bank for Reconstruction and Development (IBRD) Balance Sheet. For this example we will be using the CSV file "IBRD_Balance_Sheet__FY2010.csv" which is provided in the supplementary material for the lab. The CSV file includes records which are characterised by a Category (and subcategories), Line Item, Fiscal Year, and Amount (in US$ millions). We first start with imports:

In [44]:
# If you are using Google Colab, you would need to run the below line to install Cubes.
# You can comment the below line if you are running a local python installation with Cubes installed.
!pip install cubes
!pip install sqlalchemy==1.3.20
from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv

We can now load the data, create a table and populate it with contents of the CSV file. Note the categories and subcategories created; check the CSV file to link the below commands with the contents of the dataset.

In [45]:
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "IBRD_Balance_Sheet__FY2010.csv",
                      table_name="ibrd_balance",
                      fields=[
                          ("category", "string"),
                          ("category_label", "string"),
                          ("subcategory", "string"),
                          ("subcategory_label", "string"),
                          ("line_item", "string"),
                          ("year", "integer"),
                          ("amount", "integer")],
                      create_id=True
                     )

## 3. Creating a data cube

Everything in Cubes happens in an *analytical workspace*. It contains cubes, maintains connections to the data stores (with cube data), provides connection to external cubes and more.

The workspace properties are specified in a configuration file slicer.ini (default name). The first thing we have to do is to specify a data store – a database which will host the cube’s data:

In [46]:
from cubes import Workspace

workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")

The structure of data cubes (in terms of dimensions, measures, and aggregates) is specified in JSON files. We now import file 'tutorial_model.json' (found in the lab supplementary material) which includes an example model of the data cube, dimension tables, and aggregate functions for the CSV file we loaded previously.

In [47]:
workspace.import_model("tutorial_model.json")

**Please make sure to inspect the structure of the above JSON file**

We can now create a data cube based on the above data cube model and data table:

In [48]:
cube = workspace.cube("ibrd_balance")

## 4. Aggregations and OLAP operations

*Browser* is an object that does the actual aggregations and other data queries for a cube. To obtain one:

In [49]:
browser = workspace.browser(cube)

We can now compute aggregates of the data cube as specified by the data cube model. For computing the total count of records:

In [50]:
result = browser.aggregate()
result.summary["record_count"]

62

ERROR:sqlalchemy.pool.impl.NullPool:Exception during reset or similar
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 893, in _reset
    pool._dialect.do_rollback(self)
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/default.py", line 543, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 133354449227776 and this is thread id 133353558763072.
ERROR:sqlalchemy.pool.impl.NullPool:Exception closing connection <sqlite3.Connection object at 0x7948b3d7b740>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "/usr/local/lib/python3.10/dist-pa

For computing a sum of the amount:

In [51]:
result.summary["amount_sum"]

1116860

Now we can try to compute aggregates by year:

In [52]:
result = browser.aggregate(drilldown=["year"])
for record in result:
    print(record)

{'year': 2009, 'amount_sum': 550840, 'record_count': 31}
{'year': 2010, 'amount_sum': 566020, 'record_count': 31}


Or compute aggregates by item category:

In [53]:
result = browser.aggregate(drilldown=["item"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'amount_sum': 558430, 'record_count': 32}
{'item.category': 'e', 'item.category_label': 'Equity', 'amount_sum': 77592, 'record_count': 8}
{'item.category': 'l', 'item.category_label': 'Liabilities', 'amount_sum': 480838, 'record_count': 22}


We can also perform *slicing* and *dicing* operations on the data cube. The below example performs a slicing operation on the data cube by selecting only entries with the year being 2009, and displays aggregates according to the item category. Here, a *cell* defines a point of interest – portion of the cube to be aggergated or browsed.

In [54]:
import cubes as cubes
cuts = [cubes.PointCut("year", ["2009"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["item"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'amount_sum': 275420, 'record_count': 16}
{'item.category': 'e', 'item.category_label': 'Equity', 'amount_sum': 40037, 'record_count': 4}
{'item.category': 'l', 'item.category_label': 'Liabilities', 'amount_sum': 235383, 'record_count': 11}


It's worth noting that in Cubes, slicing operations can be created by either specifying a "point cut" which selects a single value of an attribute in a given dimension (called using the cubes.PointCut() function as above), or by specifying a "range cut", which selects a range of values for a given dimension. The range cut can be called using the cubes.RangeCut() function, which takes as input the attribute name, the minimum value of the specified range, and the maximum value of the range.

Similarly, we can perform a *dicing* operation on the data cube by performing a selection on two or more dimensions. The below example performs a dicing operation on the data cube, selecting entries with the year being 2009 and the item category being "a", and displays the aggregate results:

In [55]:
cuts = [cubes.PointCut("year", ["2009"]),cubes.PointCut("item", ["a"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell,drilldown=["item"])
result.summary

{'amount_sum': 275420, 'record_count': 16}

We can also *drill down* lower in the Category hierarchy. Here, we perform a dicing operation to select records with year being 2009 and item category being "a" (corresponding to assets), and show aggregates for each subcategory level.

In [56]:
cuts = [cubes.PointCut("year", ["2009"]),cubes.PointCut("item", ["a"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell,drilldown=["item:subcategory"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'da', 'item.subcategory_label': 'Derivative Assets', 'amount_sum': 123065, 'record_count': 4}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'dfb', 'item.subcategory_label': 'Due from Banks', 'amount_sum': 3044, 'record_count': 2}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'i', 'item.subcategory_label': 'Investments', 'amount_sum': 41012, 'record_count': 1}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'lo', 'item.subcategory_label': 'Loans Outstanding', 'amount_sum': 103657, 'record_count': 1}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'nn', 'item.subcategory_label': 'Nonnegotiable', 'amount_sum': 1202, 'record_count': 1}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'oa', 'item.subcategory_label': 'Other Assets', 'amount_sum': 2247, 'record_count': 3}
{'ite

ERROR:sqlalchemy.pool.impl.NullPool:Exception during reset or similar
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 893, in _reset
    pool._dialect.do_rollback(self)
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/default.py", line 543, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 133354449227776 and this is thread id 133353558763072.
ERROR:sqlalchemy.pool.impl.NullPool:Exception closing connection <sqlite3.Connection object at 0x7948b3d7b740>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "/usr/local/lib/python3.10/dist-pa

</br>

**Exercise**:

Using the same CSV file and data cube in the above lab tutorial: </br>
i) modify the "tutorial_model.json" file to include aggregate measures for the minimum and maximum amount in the data cube; </br>
ii) using these implemented aggregate measures, produce the values for the minimum and maximum amount in the data per year.



  